# Creating Calendar Events with PydanticAI
## A Step-by-Step Tutorial

This tutorial will show you how to create a conversational AI assistant that helps users create calendar events and generate ICS files.

### Setup and Dependencies

First, let's install the required packages and import our dependencies:

In [3]:
!uv pip install -q pydantic-ai  python-dotenv

In [4]:
!uv pip install zoneinfo

Using Python 3.11.10 environment at /Users/rianders/Documents/notebooks/.venv
  × No solution found when resolving dependencies:                                  
  ╰─▶ Because zoneinfo was not found in the package registry and you require
      zoneinfo, we can conclude that your requirements are unsatisfiable.


In [5]:
import os
from dotenv import load_dotenv
import nest_asyncio

import asyncio
from datetime import datetime, timedelta
from typing import Optional
from pydantic_ai import RunContext, Tool, Agent
from zoneinfo import ZoneInfo
import uuid

In [6]:
# Load the environment variables from the .env file
load_dotenv()

# dotenv_path = join(dirname(__file__), '.env')
# load_dotenv(dotenv_path)

os.environ["ANTHROPIC_API_KEY"] = os.environ.get("ANTHROPIC_KEY")
nest_asyncio.apply()

### Step 1: Event Management Classes

First, we'll create our event management class that will maintain the state of our calendar event:

In [7]:
class EventDetails:
    def __init__(self):
        self.summary: Optional[str] = None
        self.start_time: Optional[datetime] = None
        self.end_time: Optional[datetime] = None
        self.description: Optional[str] = None
        self.location: Optional[str] = None
        self.timezone: Optional[str] = None

    def is_complete(self) -> tuple[bool, list[str]]:
        missing = []
        if not self.summary:
            missing.append("event name/summary")
        if not self.start_time:
            missing.append("start time")
        if not self.end_time:
            missing.append("end time")
        if not self.timezone:
            missing.append("timezone")
        return len(missing) == 0, missing

    def reset(self):
        self.__init__()

# Create a global instance for our event state
current_event = EventDetails()

### Step 2: ICS File Generation

Next, let's create the function that will generate our ICS file content:

In [8]:
def create_ics_content(event: EventDetails) -> str:
    """Generate ICS file content from event details."""
    ics_content = [
        "BEGIN:VCALENDAR",
        "VERSION:2.0",
        "PRODID:-//PydanticAI//Calendar Tool//EN",
        "BEGIN:VEVENT",
        f"UID:{uuid.uuid4()}",
        f"DTSTAMP:{datetime.now().strftime('%Y%m%dT%H%M%SZ')}",
        f"DTSTART;TZID={event.timezone}:{event.start_time.strftime('%Y%m%dT%H%M%S')}",
        f"DTEND;TZID={event.timezone}:{event.end_time.strftime('%Y%m%dT%H%M%S')}",
        f"SUMMARY:{event.summary}"
    ]
    
    if event.description:
        ics_content.append(f"DESCRIPTION:{event.description}")
    if event.location:
        ics_content.append(f"LOCATION:{event.location}")
    
    ics_content.extend([
        "END:VEVENT",
        "END:VCALENDAR"
    ])
    
    return "\r\n".join(ics_content)

### Step 3: Tool Functions

Now let's create our three main tool functions:

In [9]:
async def set_event_details(
    ctx: RunContext[None],
    field: str,
    value: str
) -> str:
    """Set a specific field for the event."""
    field = field.lower()
    
    if field == "timezone":
        current_event.timezone = value
        return f"Set timezone to {value}"
    
    elif field == "summary" or field == "title":
        current_event.summary = value
        return f"Set event name to: {value}"
    
    elif field == "description":
        current_event.description = value
        return f"Set event description to: {value}"
    
    elif field == "location":
        current_event.location = value
        return f"Set event location to: {value}"
    
    elif field == "start_time":
        try:
            if not current_event.timezone:
                return "Please set the timezone first before setting times."
            
            if ":" in value:  # ISO format
                dt = datetime.fromisoformat(value)
            else:  # Natural language
                dt = datetime.now(ZoneInfo(current_event.timezone))
                if "tomorrow" in value.lower():
                    dt += timedelta(days=1)
            
            current_event.start_time = dt
            return f"Set event start time to: {dt.strftime('%Y-%m-%d %H:%M %Z')}"
        except ValueError as e:
            return f"Error parsing start time: {str(e)}"
    
    elif field == "end_time":
        try:
            if not current_event.timezone:
                return "Please set the timezone first before setting times."
            
            if ":" in value:  # ISO format
                dt = datetime.fromisoformat(value)
            else:  # Natural language
                dt = datetime.now(ZoneInfo(current_event.timezone))
                if "tomorrow" in value.lower():
                    dt += timedelta(days=1)
            
            current_event.end_time = dt
            return f"Set event end time to: {dt.strftime('%Y-%m-%d %H:%M %Z')}"
        except ValueError as e:
            return f"Error parsing end time: {str(e)}"
    
    return f"Unknown field: {field}"

async def check_event_status(ctx: RunContext[None]) -> str:
    """Check what information is still needed for the event."""
    is_complete, missing = current_event.is_complete()
    
    if is_complete:
        return "All required event details are set! You can now generate the ICS file."
    else:
        return f"Still need the following information: {', '.join(missing)}"

async def generate_ics_file(ctx: RunContext[None]) -> str:
    """Generate an ICS file from the current event details."""
    is_complete, missing = current_event.is_complete()
    
    if not is_complete:
        return f"Cannot generate ICS file yet. Missing information: {', '.join(missing)}"
    
    try:
        ics_content = create_ics_content(current_event)
        current_event.reset()  # Reset for next event
        return f"ICS file generated successfully:\n\n{ics_content}"
    except Exception as e:
        return f"Error generating ICS file: {str(e)}"

### Step 4: Creating the Tools

Let's create our tool instances:

In [10]:
# Create the tools
set_event_tool = Tool(
    set_event_details,
    name="set_event_details",
    description="Set a specific detail for the event (timezone, summary, description, location, start_time, end_time)"
)

check_status_tool = Tool(
    check_event_status,
    name="check_event_status",
    description="Check what information is still needed for the event"
)

generate_ics_tool = Tool(
    generate_ics_file,
    name="generate_ics_file",
    description="Generate an ICS file from the current event details"
)

### Step 5: Setting Up the Agent

Now we'll create our Claude agent with the appropriate tools and system prompt:

In [11]:
agent = Agent(
    "claude-3-5-sonnet-latest",
    tools=[set_event_tool, check_status_tool, generate_ics_tool],
    system_prompt="""You are a helpful assistant that creates calendar events. Guide users through creating an event by:
    1. First ask for their timezone if not set
    2. Ask for event name/summary
    3. Ask for start and end times
    4. Optionally ask for description and location
    5. Generate the ICS file when all required information is provided
    
    Use these tools:
    - set_event_details: Set specific field and value using set_event_details 
    - check_event_status: Check what information is still needed
    - generate_ics_file: Create the final ICS file
    
    Be conversational but efficient in gathering information.
    After each user response, use check_event_status to see what information is still needed.
    Make sure to use set_event_details for each piece of information the user provides."""
)

### Step 6: Using the Agent

Now let's try creating an event! Here's an example conversation flow:

In [13]:
result = await agent.run("I'm in New York")
print("Assistant:", result.data)

Assistant: Great, I've set your timezone to America/New_York. Now, what would you like to name your event? This will be the title or summary that appears on your calendar.


In [14]:
result = await agent.run("Hi, I want to create a calendar event")
print("Assistant:", result.data)

Assistant: As I thought, we'll need several pieces of information. Let's start with your timezone so we can ensure all times are properly set.


In [15]:
# Set timezone
result = await agent.run("I'm in New York")
print("Assistant:", result.data)

Assistant: Great! Now, what would you like to name your event? This will be the title that appears in your calendar.


In [16]:
# Set event details
result = await agent.run("I want to schedule a team meeting tomorrow at 2pm for one hour")
print("Assistant:", result.data)

Assistant: First, I need to know your timezone to properly set the meeting time. What timezone are you in?

Once you provide the timezone, I'll help set up the team meeting for tomorrow at 2 PM for one hour. I'll also add "Team Meeting" as the summary unless you'd like a different title for the meeting.

Please let me know:
1. Your timezone
2. If you'd like a different title than "Team Meeting"


In [17]:
# Add location
result = await agent.run("It's in the conference room")
print("Assistant:", result.data)

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': "messages.2: the following `tool_use` ids were not found in `tool_result` blocks: {'toolu_01MPskBE38n72QSePEQ2AoRE'}"}}

In [18]:
# Generate ICS file
result = await agent.run("That's all the details, please create the calendar event")
print("Assistant:", result.data)

Assistant: I see we don't have any event details yet! Let's start from the beginning. First, I need to know your timezone to ensure the event times are set correctly.

What timezone are you in? (For example: America/New_York, Europe/London, etc.)


In [19]:
# Start the conversation










### Try it yourself!

Now you can try creating your own calendar events. Start with:

In [ ]:
result = await agent.run("I want to create a calendar event")
print("Assistant:", result.data)

# Then respond to the assistant's questions...

### Tips for Usage:

1. Always start by providing your timezone
2. Be specific about dates and times
3. You can provide multiple pieces of information in one message
4. Use the generated ICS content with any calendar application that supports iCalendar format

### Common Questions:

**Q: What time formats are supported?**
A: The tool supports both ISO format (e.g., "2024-01-04T14:00:00") and some natural language (e.g., "tomorrow at 2pm").

**Q: What information is required?**
A: Required fields are: timezone, summary (event name), start time, and end time. Description and location are optional.

**Q: Can I modify an event after starting to create it?**
A: Yes, you can update any field by providing new information before generating the ICS file.

### Next Steps

You can enhance this tutorial by:
1. Adding more natural language time parsing
2. Supporting recurring events
3. Adding validation for business hours
4. Implementing calendar checking for conflicts
5. Adding support for attendees and reminders